##Sentiment analysis of IMDB movie review dataset

In [1]:
%tensorflow_version 2.0

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


Importing the libraries and loading the data sets

In [2]:
import tensorflow as tf
import keras
import numpy as np
import os

from keras.datasets import imdb
from keras.preprocessing import sequence

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

17464789/17464789 [==============================] - 0s 0us/step


In [3]:
(train_data[1])

[1,
 194,
 1153,
 194,
 8255,
 78,
 228,
 5,
 6,
 1463,
 4369,
 5012,
 134,
 26,
 4,
 715,
 8,
 118,
 1634,
 14,
 394,
 20,
 13,
 119,
 954,
 189,
 102,
 5,
 207,
 110,
 3103,
 21,
 14,
 69,
 188,
 8,
 30,
 23,
 7,
 4,
 249,
 126,
 93,
 4,
 114,
 9,
 2300,
 1523,
 5,
 647,
 4,
 116,
 9,
 35,
 8163,
 4,
 229,
 9,
 340,
 1322,
 4,
 118,
 9,
 4,
 130,
 4901,
 19,
 4,
 1002,
 5,
 89,
 29,
 952,
 46,
 37,
 4,
 455,
 9,
 45,
 43,
 38,
 1543,
 1905,
 398,
 4,
 1649,
 26,
 6853,
 5,
 163,
 11,
 3215,
 10156,
 4,
 1153,
 9,
 194,
 775,
 7,
 8255,
 11596,
 349,
 2637,
 148,
 605,
 15358,
 8003,
 15,
 123,
 125,
 68,
 23141,
 6853,
 15,
 349,
 165,
 4362,
 98,
 5,
 4,
 228,
 9,
 43,
 36893,
 1157,
 15,
 299,
 120,
 5,
 120,
 174,
 11,
 220,
 175,
 136,
 50,
 9,
 4373,
 228,
 8255,
 5,
 25249,
 656,
 245,
 2350,
 5,
 4,
 9837,
 131,
 152,
 491,
 18,
 46151,
 32,
 7464,
 1212,
 14,
 9,
 6,
 371,
 78,
 22,
 625,
 64,
 1382,
 9,
 8,
 168,
 145,
 23,
 4,
 1690,
 15,
 16,
 4,
 1355,
 5,
 28,
 6,
 52,
 

We will have to process to make the same length of 250 words

In [4]:
train_data = keras.utils.pad_sequences(train_data, MAXLEN)
test_data = keras.utils.pad_sequences(test_data, MAXLEN)

###Creating the model
- First layer in embedding layer
- Second layer in LSTM layer
- Last layer is input layer

In [5]:
model = tf.keras.Sequential([
                            tf.keras.layers.Embedding(VOCAB_SIZE, 32),
                            tf.keras.layers.LSTM(32),
                            tf.keras.layers.Dense(1, activation = "sigmoid")
])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


###Training the model
We have to compile and train/ fit the model now

In [8]:
model.compile(loss= "binary_crossentropy", optimizer = "rmsprop",metrics =["accuracy"])

history = model.fit(train_data , train_labels , epochs = 10 , validation_split = 0.2)

Epoch 1/10
625/625 [==============================] - 71s 102ms/step - loss: 0.4498 - accuracy: 0.7829 - val_loss: 0.3283 - val_accuracy: 0.8632
Epoch 2/10
625/625 [==============================] - 23s 38ms/step - loss: 0.2618 - accuracy: 0.9004 - val_loss: 0.3369 - val_accuracy: 0.8740
Epoch 3/10
625/625 [==============================] - 15s 25ms/step - loss: 0.2020 - accuracy: 0.9265 - val_loss: 0.2921 - val_accuracy: 0.8864
Epoch 4/10
625/625 [==============================] - 10s 16ms/step - loss: 0.1634 - accuracy: 0.9420 - val_loss: 0.2876 - val_accuracy: 0.8838
Epoch 5/10
625/625 [==============================] - 8s 13ms/step - loss: 0.1366 - accuracy: 0.9530 - val_loss: 0.3108 - val_accuracy: 0.8828
Epoch 6/10
625/625 [==============================] - 9s 14ms/step - loss: 0.1172 - accuracy: 0.9591 - val_loss: 0.4726 - val_accuracy: 0.8376
Epoch 7/10
625/625 [==============================] - 9s 15ms/step - loss: 0.0980 - accuracy: 0.9687 - val_loss: 0.3904 - val_accuracy: 0

In [9]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 4s 5ms/step - loss: 0.5118 - accuracy: 0.8615
[0.5118353962898254, 0.8614799976348877]


###Making Predictions
we need to encode our review as per the dataset encoding

In [10]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return keras.utils.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)

1641221/1641221 [==============================] - 0s 0us/step
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 

Creating a decode function

In [11]:
reverese_word_index = {value: key for (key,value) in word_index.items()}
def decode_integers(integers):
  PAD =0
  text =""
  for num in integers:
    if num!= PAD:
      text += reverse_word_index[num] + " "

    return text[:-1]

  print(decode_integers(encoded))

Making predictions

In [17]:
def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred)
  print(result[0])

positive_review ="The movie was really good, loved it all the way through also the ending"
predict(positive_review)

negative_review =" that movie was worst , very bad "
predict(negative_review)

1/1 [==============================] - 0s 22ms/step
[0.9455236]
1/1 [==============================] - 0s 21ms/step
[0.642316]
